In [123]:
from IPython.display import clear_output
!pip install requests
clear_output()
import requests
import time
from bs4 import BeautifulSoup

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd

import json

In [139]:
years = list(range(1980,2024))
# URL we want to scrape. I replace the year with brackets. 
# This will allow us to replace them with each year we want to scrape
url_start = "https://www.basketball-reference.com/awards/awards_{}.html"

year = [2024]
for year in years: 
    url = url_start.format(year)  # This replace {} with the specific year
    data = requests.get(url)      # This enables us to download the web page into data (html code)

    filepath = f'/Users/cb/src/nba_mvp_ml/data/html/{year}.html'
    
    with open(filepath, "w+") as f:  # Finally, we create one specific .html file for each
        f.write(data.text)                              # of the years we want to scrape

    time.sleep(10)

In [140]:
import os

def rename_files_with_previous_year(directory):
    """
    Renames files in the specified directory by appending the previous year to the current year in the filename.
    For example, '2024.html' becomes '2023-24.html'.
    
    Parameters:
    - directory (str): Path to the directory containing the files.
    """
    # List all files in the directory
    files = os.listdir(directory)
    
    for file in files:
        # Ensure we only process files with '.html' extension
        if file.endswith('.html'):
            try:
                # Extract the year from the filename
                current_year = int(file.split('.')[0])
                previous_year = current_year - 1
                
                # Construct the new filename
                new_filename = f"{previous_year}-{str(current_year)[-2:]}.html"
                
                # Rename the file
                os.rename(
                    os.path.join(directory, file),
                    os.path.join(directory, new_filename)
                )
                print(f"Renamed: {file} -> {new_filename}")
            except ValueError:
                print(f"Skipped: {file} (not a valid year format)")

# Example usage
rename_files_with_previous_year('/Users/cb/src/nba_mvp_ml/data/html/mvp')

Renamed: 2021.html -> 2020-21.html
Renamed: 2017.html -> 2016-17.html
Renamed: 2001.html -> 2000-01.html
Renamed: 1986.html -> 1985-86.html
Renamed: 1990.html -> 1989-90.html
Renamed: 1991.html -> 1990-91.html
Renamed: 1987.html -> 1986-87.html
Renamed: 2000.html -> 1999-00.html
Renamed: 2016.html -> 2015-16.html
Renamed: 2020.html -> 2019-20.html
Renamed: 2011.html -> 2010-11.html
Renamed: 2007.html -> 2006-07.html
Renamed: 1980.html -> 1979-80.html
Renamed: 1996.html -> 1995-96.html
Renamed: 1997.html -> 1996-97.html
Renamed: 1981.html -> 1980-81.html
Renamed: 2006.html -> 2005-06.html
Renamed: 2010.html -> 2009-10.html
Renamed: 1994.html -> 1993-94.html
Renamed: 1982.html -> 1981-82.html
Renamed: 2005.html -> 2004-05.html
Renamed: 2013.html -> 2012-13.html
Renamed: 1998.html -> 1997-98.html
Renamed: 2009.html -> 2008-09.html
Renamed: 2008.html -> 2007-08.html
Renamed: 2024.html -> 2023-24.html
Renamed: 1999.html -> 1998-99.html
Renamed: 2012.html -> 2011-12.html
Renamed: 2004.html -

In [7]:
with open(filepath) as f:
    page = f.read()

#page 
soup = BeautifulSoup(page, "html.parser")

# The header of the table is useless for us. Thus, it'll be the first thing we are going to erase
soup.find('tr', class_="over_header").decompose()
# As we just want the first table, let's get rid of the others. We'll do this by finding out the concrete table 
# we want to scrape. In order to do that, we search its 'id'. That's unique in the whole .html file. 
mvp_table = soup.find(id="mvp")

In [9]:
# Pandas is capable of reading tables in html

!pip install lxml
clear_output()

In [10]:
# mvp_1991 = pd.read_html(str(mvp_table))   Argument must be a string
# mvp_1991  It's not a DataFrame. Instead, it's a list of them.  
mvp_1991 = pd.read_html(str(mvp_table))[0]    # Argument must be a string
mvp_1991.head()

/var/folders/0c/zn_s4ngs4tsbd8lhp838h4ph0000gn/T/ipykernel_12333/3313577371.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  mvp_1991 = pd.read_html(str(mvp_table))[0]    # Argument must be a string


,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48
0,1,Joel Embiid,28,PHI,73,915,1000,0.915,66,34.6,33.1,10.2,4.2,1.0,1.7,0.548,0.330,0.857,12.3,0.259
1,2,Nikola JokiÄ,27,DEN,15,674,1000,0.674,69,33.7,24.5,11.8,9.8,1.3,0.7,0.632,0.383,0.822,14.9,0.308
2,3,Giannis Antetokounmpo,28,MIL,12,606,1000,0.606,63,32.1,31.1,11.8,5.7,0.8,0.8,0.553,0.275,0.645,8.6,0.204
3,4,Jayson Tatum,24,BOS,0,280,1000,0.280,74,36.9,30.1,8.8,4.6,1.1,0.7,0.466,0.350,0.854,10.5,0.185
4,5,Shai Gilgeous-Alexander,24,OKC,0,46,1000,0.046,68,35.5,31.4,4.8,5.5,1.6,1.0,0.510,0.345,0.905,11.4,0.226


In [22]:
url2 = 'https://www.basketball-reference.com/leagues/NBA_2022.html'

data2 = requests.get(url2)      # This enables us to download the web page into data (html code)

filepath = f'/Users/cb/src/nba_mvp_ml/data/html/NBA_2022.html'

with open(filepath, "w+") as f:  # Finally, we create one specific .html file for each
    f.write(data2.text)                              # of the years we want to scrape

time.sleep(10)

In [2]:
import requests
import os
import time

import pandas as pd
from bs4 import BeautifulSoup

In [3]:

# Table 0: ID = divs_standings_E, Summary = None
# Table 1: ID = divs_standings_W, Summary = None
# Table 2: ID = per_game-team, Summary = None
# Table 3: ID = per_game-opponent, Summary = None
# Table 4: ID = totals-team, Summary = None
# Table 5: ID = totals-opponent, Summary = None
# Table 6: ID = per_poss-team, Summary = None
# Table 7: ID = per_poss-opponent, Summary = None
# Table 8: ID = advanced-team, Summary = None
# Table 9: ID = shooting-team, Summary = None
# Table 10: ID = shooting-opponent, Summary = None

In [7]:

# Function to fetch HTML files for each season
def fetch_html_files(start_year, end_year, output_dir):
    base_url = "https://www.basketball-reference.com/leagues/NBA_{}.html"
    os.makedirs(output_dir, exist_ok=True)
    
    for year in range(start_year, end_year + 1):
        url = base_url.format(year)
        filepath = os.path.join(output_dir, f"NBA_{year}.html")
        
        if not os.path.exists(filepath):  # Avoid re-downloading
            try:
                response = requests.get(url)
                response.raise_for_status()  # Raise error for failed requests
                with open(filepath, "w+", encoding="utf-8") as file:
                    file.write(response.text)
                print(f"Successfully fetched data for {year}")
            except Exception as e:
                print(f"Failed to fetch data for {year}: {e}")
            time.sleep(5)  # Delay to avoid bot detection


def fetch_advanced_player_stats(start_year, end_year, output_dir):
    """
    Fetches HTML files for player advanced stats from Basketball Reference 
    for each NBA season between start_year and end_year.

    Parameters:
    - start_year (int): The starting season year.
    - end_year (int): The ending season year.
    - output_dir (str): The directory to save the fetched HTML files.
    """
    # Base URL for advanced player stats
    base_url = "https://www.basketball-reference.com/leagues/NBA_{}_advanced.html"
    os.makedirs(output_dir, exist_ok=True)  # Ensure output directory exists
    
    for year in range(start_year, end_year + 1):
        url = base_url.format(year)
        filepath = os.path.join(output_dir, f"player_adv_{year}.html")
        
        if not os.path.exists(filepath):  # Avoid re-downloading
            try:
                response = requests.get(url)
                response.raise_for_status()  # Raise error for failed requests
                with open(filepath, "w+", encoding="utf-8") as file:
                    file.write(response.text)
                print(f"Successfully fetched advanced stats for {year}")
            except Exception as e:
                print(f"Failed to fetch advanced stats for {year}: {e}")
            time.sleep(5)  # Delay to avoid bot detection

# Function to parse HTML and extract relevant data
def parse_and_save_data(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    files = [f for f in os.listdir(input_dir) if f.endswith(".html")]
    
    for file in files:
        year = file.split("_")[1].split(".")[0]
        filepath = os.path.join(input_dir, file)
        
        with open(filepath, "r", encoding="utf-8") as html_file:
            soup = BeautifulSoup(html_file, "html.parser")

            # Print out the tables available for parsing in the html
            tables = soup.find_all("table")
            for i, table in enumerate(tables):
                print(f"Table {i}: ID = {table.get('id')}, Summary = {table.get('summary')}")
            
            # Extract team stats tables (Traditional Stats)
            traditional_per_game_table = soup.find("table", {"id": "per_game-team"})
            traditional_per_game_opp_table = soup.find("table", {"id": "per_game-opponent"})
            traditional_total_table = soup.find("table", {"id": "totals-team"})
            traditional_total_opp_table = soup.find("table", {"id": "totals-opponent"})
            advanced_table = soup.find("table", {"id": "advanced-team"})
            standings_table_east = soup.find("table", {"id": "divs_standings_E"})
            standings_table_west = soup.find("table", {"id": "divs_standings_W"})

            stat_tables = [traditional_per_game_table,traditional_per_game_opp_table, traditional_total_table,
                    traditional_total_opp_table,advanced_table, standings_table_east, standings_table_west]
            
            if all(stat_tables):
                # Parse Traditional total Stats
                traditional_total_df = pd.read_html(str(traditional_total_table))[0]
                traditional_total_df["Year"] = year

                # Parse Traditional total opponent Stats
                traditional_total_opp_df = pd.read_html(str(traditional_total_opp_table))[0]
                traditional_total_opp_df["Year"] = year

                # Parse Traditional per-game Stats
                traditional_per_game_df = pd.read_html(str(traditional_per_game_table))[0]
                traditional_per_game_df["Year"] = year

                # Parse Traditional per-game opponent Stats
                traditional_per_game_opp_df = pd.read_html(str(traditional_per_game_opp_table))[0]
                traditional_per_game_opp_df["Year"] = year

                total_df = traditional_total_df.merge(traditional_total_opp_df, on=["Team", "Year"], suffixes=("", "_opp"))
                per_game_df = traditional_per_game_df.merge(traditional_per_game_opp_df, on=["Team", "Year"], suffixes=("", "_opp"))
                traditional_df = total_df.merge(per_game_df, on=["Team", "Year"], suffixes=("", "_pg"))
                
                # Parse Advanced Stats
                advanced_df = pd.read_html(str(advanced_table))[0]
                advanced_df.columns = advanced_df.columns.get_level_values(1)
                advanced_df["Year"] = year

                # Parse Standings (Win-Loss Metrics)
                standings_df_east = pd.read_html(str(standings_table_east))[0]
                standings_df_east["Year"] = year
                standings_df_east.rename(columns={'Eastern Conference': 'Team'}, inplace=True)

                # Parse Standings (Win-Loss Metrics)
                standings_df_west = pd.read_html(str(standings_table_west))[0]
                standings_df_west["Year"] = year
                standings_df_west.rename(columns={'Western Conference': 'Team'}, inplace=True)

                # print(traditional_df.columns)
                # print(advanced_df.columns)
                # print(standings_df_east.columns)
                # print(standings_df_west.columns)
                
                # Merge the data
                team_data = traditional_df.merge(advanced_df, on=["Team", "Year"], suffixes=("_trad", "_adv"))
                standings_df = pd.concat([standings_df_east, standings_df_west], axis=0)
                # standings_df = standings_df_east.merge(standings_df_west, on=["Year"])
                team_data = team_data.merge(standings_df, on=["Team", "Year"], suffixes=("", "_wl"))
                team_data['Team'] = team_data['Team'].str.replace('*', '', regex=False)
                
                # Save the team stats for this year
                output_path = os.path.join(output_dir, f"NBA_Teams_{year}.csv")
                team_data.to_csv(output_path, index=False)
                print(f"Saved team stats for {year} to {output_path}")
            else:
                print(f"Missing tables for {year}")


def update_team_data(year, stats_dir, teams_dir, output_dir, teams_json):
    """
    Updates NBA team data by inserting team abbreviations and IDs and renaming the 'Year' column.
    
    Parameters:
    - year: int, the season year to process (e.g., 2023)
    - stats_dir: str, folder path containing stats CSV files
    - teams_dir: str, folder path containing teams CSV files
    - output_dir: str, folder path to save updated teams CSV files
    """
    # Define file paths
    stats_file = os.path.join(stats_dir, f"stats_{year}.csv")
    teams_file = os.path.join(teams_dir, f"NBA_Teams_{year}.csv")
    output_file = os.path.join(output_dir, f"team_stats_{year}_updated.csv")
    
    # Load the stats and teams data
    stats_df = pd.read_csv(stats_file)
    teams_df = pd.read_csv(teams_file)
    
    # Load team abbreviation data from teams.json
    with open(teams_json, 'r', encoding='utf-8') as json_file:
        team_data = json.load(json_file)
    
    # Create a mapping from team names to abbreviations and IDs
    team_mapping = {team["teamName"]: {"abbreviation": team["abbreviation"], "id": team["teamId"]} for team in team_data}
    
    # Rename the 'Year' column to 'SEASON_ID'
    if 'Year' in teams_df.columns:
        teams_df.rename(columns={'Year': 'SEASON_ID'}, inplace=True)
    
    # Remove asterisks from team names in the "Team" column
    teams_df['Team'] = teams_df['Team'].str.replace('*', '', regex=False).str.strip()
    
    # Map TEAM_ABBREVIATION and TEAM_ID from team_mapping
    teams_df['TEAM_ABBREVIATION'] = teams_df['Team'].map(lambda x: team_mapping.get(x, {}).get('abbreviation', None))
    teams_df['TEAM_ID'] = teams_df['Team'].map(lambda x: team_mapping.get(x, {}).get('id', None))
    
    # Error handling: Check for mismatches
    missing_teams = teams_df[teams_df['TEAM_ABBREVIATION'].isnull() | teams_df['TEAM_ID'].isnull()]
    if not missing_teams.empty:
        print(f"Warning: The following teams could not be matched for {year}:")
        print(missing_teams[['Team']])
    
    # Merge team data with player stats based on TEAM_ABBREVIATION
    try:
        stats_team_data = stats_df[['TEAM_ABBREVIATION', 'TEAM_ID']].drop_duplicates()
        teams_df = teams_df.merge(stats_team_data, on=['TEAM_ABBREVIATION', 'TEAM_ID'], how='left')
    except KeyError as e:
        print(f"Merge Error for {year}: {e}")
        return
    
    # Save the updated DataFrame
    teams_df.to_csv(output_file, index=False)
    print(f"Updated teams data saved to {output_file}")

In [134]:
stats_dir = '/Users/cb/src/nba_mvp_ml/data/processed/by_season/players'
teams_dir = '/Users/cb/src/nba_mvp_ml/data/raw/basketball-reference'
output_dir = '/Users/cb/src/nba_mvp_ml/data/processed/by_season/team (basketball-reference)'
teams_json = '/Users/cb/src/nba_mvp_ml/data/teams.json'

for year in list(range(1980,2024)):
    update_team_data(year, stats_dir, teams_dir, output_dir, teams_json)

Updated teams data saved to /Users/cb/src/nba_mvp_ml/data/processed/by_season/team (basketball-reference)/team_stats_1980_updated.csv
Updated teams data saved to /Users/cb/src/nba_mvp_ml/data/processed/by_season/team (basketball-reference)/team_stats_1981_updated.csv
Updated teams data saved to /Users/cb/src/nba_mvp_ml/data/processed/by_season/team (basketball-reference)/team_stats_1982_updated.csv
Updated teams data saved to /Users/cb/src/nba_mvp_ml/data/processed/by_season/team (basketball-reference)/team_stats_1983_updated.csv
Updated teams data saved to /Users/cb/src/nba_mvp_ml/data/processed/by_season/team (basketball-reference)/team_stats_1984_updated.csv
Updated teams data saved to /Users/cb/src/nba_mvp_ml/data/processed/by_season/team (basketball-reference)/team_stats_1985_updated.csv
Updated teams data saved to /Users/cb/src/nba_mvp_ml/data/processed/by_season/team (basketball-reference)/team_stats_1986_updated.csv
Updated teams data saved to /Users/cb/src/nba_mvp_ml/data/proc

In [121]:
# # Directory to save the HTML files
# html_dir = '/Users/cb/src/nba_mvp_ml/data/html'
# os.makedirs(html_dir, exist_ok=True)

# # Base URL for season standings
# base_url = "https://www.basketball-reference.com/leagues/NBA_{}.html"

# # Download HTML files for seasons 1980 to 2023
# for year in range(1980, 2024):
#     url = base_url.format(year)
#     filepath = os.path.join(html_dir, f"NBA_{year}.html")
#     if not os.path.exists(filepath):  # Avoid re-downloading if file exists
#         try:
#             response = requests.get(url)
#             if response.status_code == 200:
#                 with open(filepath, "w", encoding="utf-8") as f:
#                     f.write(response.text)
#                 print(f"Saved HTML for {year}")
#             else:
#                 print(f"Failed to fetch {url}: {response.status_code}")
#             time.sleep(4)  # Delay to avoid bot detection
#         except Exception as e:
#             print(f"Error fetching {url}: {e}")

In [ ]:
# ADVANCED PLAYER STATS
output_directory = "/Users/cb/src/nba_mvp_ml/data/html/player-stats"
fetch_advanced_player_stats(1980, 2024, output_directory)

Successfully fetched advanced stats for 1980


In [9]:
import os

def rename_files_one_year_earlier(directory):
    """
    Renames files in the specified directory, decreasing the year in the filename by one.
    
    Parameters:
    - directory (str): Path to the directory containing the files to rename.
    """
    for file_name in sorted(os.listdir(directory)):
        if file_name.endswith(".html"):
            try:
                # Extract the year from the filename
                parts = file_name.split("_")
                if len(parts) > 1 and parts[-1].endswith(".html"):
                    year_part = parts[-1].split(".")[0]  # Get the year without the extension
                    new_year = int(year_part) - 1        # Decrease the year by 1
                    
                    # Build the new filename
                    new_file_name = file_name.replace(year_part, str(new_year))
                    old_path = os.path.join(directory, file_name)
                    new_path = os.path.join(directory, new_file_name)
                    
                    # Rename the file
                    os.rename(old_path, new_path)
                    print(f"Renamed: {file_name} -> {new_file_name}")
            except Exception as e:
                print(f"Error renaming {file_name}: {e}")

# Example usage
directory_path = "/Users/cb/src/nba_mvp_ml/data/html/player-adv-stats"
rename_files_one_year_earlier(directory_path)

Renamed: player_adv_1980.html -> player_adv_1979.html
Renamed: player_adv_1981.html -> player_adv_1980.html
Renamed: player_adv_1982.html -> player_adv_1981.html
Renamed: player_adv_1983.html -> player_adv_1982.html
Renamed: player_adv_1984.html -> player_adv_1983.html
Renamed: player_adv_1985.html -> player_adv_1984.html
Renamed: player_adv_1986.html -> player_adv_1985.html
Renamed: player_adv_1987.html -> player_adv_1986.html
Renamed: player_adv_1988.html -> player_adv_1987.html
Renamed: player_adv_1989.html -> player_adv_1988.html
Renamed: player_adv_1990.html -> player_adv_1989.html
Renamed: player_adv_1991.html -> player_adv_1990.html
Renamed: player_adv_1992.html -> player_adv_1991.html
Renamed: player_adv_1993.html -> player_adv_1992.html
Renamed: player_adv_1994.html -> player_adv_1993.html
Renamed: player_adv_1995.html -> player_adv_1994.html
Renamed: player_adv_1996.html -> player_adv_1995.html
Renamed: player_adv_1997.html -> player_adv_1996.html
Renamed: player_adv_1998.htm

In [12]:
import os
import pandas as pd
from bs4 import BeautifulSoup

def parse_and_save_player_advanced_stats(input_dir, output_dir):
    """
    Parses HTML files containing player advanced stats and saves them as CSV files.

    Parameters:
    - input_dir (str): Directory containing the input HTML files.
    - output_dir (str): Directory to save the output CSV files.
    """
    os.makedirs(output_dir, exist_ok=True)
    files = [f for f in os.listdir(input_dir) if f.endswith(".html")]
    
    for file in files:
        year = file.split("_")[-1].split(".")[0]
        filepath = os.path.join(input_dir, file)
        
        with open(filepath, "r", encoding="utf-8") as html_file:
            soup = BeautifulSoup(html_file, "html.parser")
            
            # Locate the advanced stats table
            advanced_table = soup.find("table", {"id": "advanced"})
            
            if advanced_table:
                try:
                    # Parse the table into a DataFrame
                    advanced_df = pd.read_html(str(advanced_table))[0]
                    advanced_df["Year"] = year
                    
                    # Clean up column names
                    advanced_df.columns = [col.replace(" ", "_").replace("%", "Pct") for col in advanced_df.columns]
                    
                    # Save to CSV
                    output_path = os.path.join(output_dir, f"player-adv-{year}.csv")
                    advanced_df.to_csv(output_path, index=False)
                    print(f"Saved player advanced stats for {year} to {output_path}")
                except Exception as e:
                    print(f"Failed to parse advanced stats table for {year}: {e}")
            else:
                print(f"No advanced stats table found for {year}")

parse_and_save_player_advanced_stats(input_dir='/Users/cb/src/nba_mvp_ml/data/html/player-adv-stats',output_dir='/Users/cb/src/nba_mvp_ml/data/raw/basketball-reference/player-adv-stats')

/var/folders/0c/zn_s4ngs4tsbd8lhp838h4ph0000gn/T/ipykernel_38397/1526506079.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  advanced_df = pd.read_html(str(advanced_table))[0]


Saved player advanced stats for 1987 to /Users/cb/src/nba_mvp_ml/data/raw/basketball-reference/player-adv-stats/player-adv-1987.csv


/var/folders/0c/zn_s4ngs4tsbd8lhp838h4ph0000gn/T/ipykernel_38397/1526506079.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  advanced_df = pd.read_html(str(advanced_table))[0]


Saved player advanced stats for 1991 to /Users/cb/src/nba_mvp_ml/data/raw/basketball-reference/player-adv-stats/player-adv-1991.csv


/var/folders/0c/zn_s4ngs4tsbd8lhp838h4ph0000gn/T/ipykernel_38397/1526506079.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  advanced_df = pd.read_html(str(advanced_table))[0]


...

Saved player advanced stats for 1992 to /Users/cb/src/nba_mvp_ml/data/raw/basketball-reference/player-adv-stats/player-adv-1992.csv


/var/folders/0c/zn_s4ngs4tsbd8lhp838h4ph0000gn/T/ipykernel_38397/1526506079.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  advanced_df = pd.read_html(str(advanced_table))[0]


Saved player advanced stats for 2015 to /Users/cb/src/nba_mvp_ml/data/raw/basketball-reference/player-adv-stats/player-adv-2015.csv


/var/folders/0c/zn_s4ngs4tsbd8lhp838h4ph0000gn/T/ipykernel_38397/1526506079.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  advanced_df = pd.read_html(str(advanced_table))[0]


Saved player advanced stats for 2003 to /Users/cb/src/nba_mvp_ml/data/raw/basketball-reference/player-adv-stats/player-adv-2003.csv


In [5]:

# Directory to save the CSV files
csv_dir = '/Users/cb/src/nba_mvp_ml/data/raw/basketball-reference/player-adv-stats'
os.makedirs(csv_dir, exist_ok=True)

# Directory containing the downloaded HTML files
html_dir = '/Users/cb/src/nba_mvp_ml/data/html/player-adv-stats'

parse_and_save_data(input_dir=html_dir, output_dir=csv_dir)

Table 0: ID = advanced, Summary = None
Table 1: ID = advanced_post, Summary = None
Missing tables for adv
Table 0: ID = advanced, Summary = None
Table 1: ID = advanced_post, Summary = None
Missing tables for adv
Table 0: ID = advanced, Summary = None
Table 1: ID = advanced_post, Summary = None
Missing tables for adv
Table 0: ID = advanced, Summary = None
Table 1: ID = advanced_post, Summary = None
Missing tables for adv
Table 0: ID = advanced, Summary = None
Table 1: ID = advanced_post, Summary = None
Missing tables for adv
Table 0: ID = advanced, Summary = None
Table 1: ID = advanced_post, Summary = None
Missing tables for adv
Table 0: ID = advanced, Summary = None
Table 1: ID = advanced_post, Summary = None
Missing tables for adv
Table 0: ID = advanced, Summary = None
Table 1: ID = advanced_post, Summary = None
Missing tables for adv
Table 0: ID = advanced, Summary = None
Table 1: ID = advanced_post, Summary = None
Missing tables for adv
Table 0: ID = advanced, Summary = None
Table 1

In [135]:
# # Path to the existing teams.json file
# json_file_path = '/Users/cb/src/nba_mvp_ml/data/teams.json'

# # Corrected data for the historical teams with the new TEAM_IDs
# historical_teams = [
#     {"teamName": "Seattle SuperSonics", "abbreviation": "SEA", "teamId": 1610612760},
#     {"teamName": "New Jersey Nets", "abbreviation": "NJN", "teamId": 1610612751},
#     {"teamName": "Kansas City Kings", "abbreviation": "KCK", "teamId": 1610612739},
#     {"teamName": "San Diego Clippers", "abbreviation": "SDC", "teamId": 1610612742},
#     {"teamName": "Washington Bullets", "abbreviation": "WSB", "teamId": 1610612764},
#     {"teamName": "Vancouver Grizzlies", "abbreviation": "VAN", "teamId": 1610612763},
#     {"teamName": "New Orleans Hornets", "abbreviation": "NOH", "teamId": 1610612740},
#     {"teamName": "Charlotte Bobcats", "abbreviation": "CHA", "teamId": 1610612766},
#     {"teamName": "New Orleans/Oklahoma City Hornets", "abbreviation": "NOK", "teamId": 1610612740}
# ]

# # Load the existing teams.json file
# with open(json_file_path, 'r', encoding='utf-8') as f:
#     teams_data = json.load(f)

# # Update or add historical team data
# existing_team_names = {team["teamName"] for team in teams_data}

# for new_team in historical_teams:
#     if new_team["teamName"] in existing_team_names:
#         # Update existing team entry
#         for team in teams_data:
#             if team["teamName"] == new_team["teamName"]:
#                 team.update(new_team)
#                 break
#     else:
#         # Add new team entry
#         teams_data.append(new_team)

# # Save the updated teams.json file
# with open(json_file_path, 'w', encoding='utf-8') as f:
#     json.dump(teams_data, f, indent=4, ensure_ascii=False)

# print("teams.json has been successfully updated with historical teams.")

In [145]:
import pandas as pd

def extract_mvp_voting_table(html_file, output_csv):
    # Read the HTML file
    with open(html_file, 'r', encoding='utf-8') as file:
        content = file.read()
    
    # Use pandas to extract tables
    tables = pd.read_html(content)
    
    # Identify the MVP voting table by inspecting the first few rows of all tables
    # for i, table in enumerate(tables):
    #     print(f"Table {i}:\n{table.head()}\n")
    
    # Assuming the MVP voting table is the first one (adjust the index if necessary)
    mvp_voting_table = tables[0]
    mvp_voting_table.columns = mvp_voting_table.columns.get_level_values(1)
    
    # Save the MVP voting table to CSV
    mvp_voting_table.to_csv(output_csv, index=False)
    print(f"MVP voting table saved to {output_csv}")

# Example usage


for year in list(range(1980, 2024)):
    html_file = f'/Users/cb/src/nba_mvp_ml/data/html/mvp/{year}-{str(year+1)[2:]}.html'
    output_csv = f'/Users/cb/src/nba_mvp_ml/data/processed/by_season/mvp/mvp_{year}-{str(year+1)[2:]}.csv'
    extract_mvp_voting_table(html_file, output_csv)

Table 0:
  Unnamed: 0_level_0   Unnamed: 1_level_0 Unnamed: 2_level_0  \
                Rank               Player                Age   
0                  1        Julius Erving                 30   
1                  2           Larry Bird                 24   
2                  3  Kareem Abdul-Jabbar                 33   
3                  4         Moses Malone                 25   
4                  5        George Gervin                 28   

  Unnamed: 3_level_0 Voting                        Unnamed: 8_level_0  \
                  Tm  First Pts Won Pts Max  Share                  G   
0                PHI     28     454     690  0.658                 82   
1                BOS     20     423     690  0.613                 82   
2                LAL      8     286     690  0.414                 80   
3                HOU      8     180     690  0.261                 80   
4                SAS      1      83     690  0.120                 82   

  Per Game                    

In [13]:
pd.read_csv('/Users/cb/src/nba_mvp_ml/data/raw/basketball-reference/player-adv-stats/player-adv-2023.csv').columns

Index(['Rk', 'Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'PER', 'TSPct',
       '3PAr', 'FTr', 'ORBPct', 'DRBPct', 'TRBPct', 'ASTPct', 'STLPct',
       'BLKPct', 'TOVPct', 'USGPct', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM',
       'DBPM', 'BPM', 'VORP', 'Awards', 'Year'],
      dtype='object')

In [14]:
pd.read_csv('/Users/cb/src/nba_mvp_ml/data/processed/by_season/players-deprecated/stats_2023.csv').columns

Index(['Unnamed: 0', 'PLAYER_ID', 'PLAYER_FULLNAME', 'SEASON_ID', 'LEAGUE_ID',
       'TEAM_ID', 'TEAM_ABBREVIATION', 'PLAYER_AGE', 'GP', 'GS', 'MIN', 'FGM',
       'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
       'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'MIN_PG', 'FGM_PG', 'FGA_PG', 'FG_PCT_PG', 'FG3M_PG', 'FG3A_PG',
       'FG3_PCT_PG', 'FTM_PG', 'FTA_PG', 'FT_PCT_PG', 'OREB_PG', 'DREB_PG',
       'REB_PG', 'AST_PG', 'STL_PG', 'BLK_PG', 'TOV_PG', 'PF_PG', 'PTS_PG',
       'TS%', 'eFG%', 'PER', 'WS'],
      dtype='object')

In [47]:
import os
import pandas as pd

def fix_encoding(text):
        try:
            return text.encode('latin1').decode('utf-8').upper()
        except (UnicodeEncodeError, UnicodeDecodeError):
            return text.upper()  # Return original text if fixing fails
            
def merge_player_stats(year, adv_dir, stats_dir, output_dir):
    """
    Merges advanced stats and traditional stats for a given year.

    Parameters:
    - year (int): The year for which data is being merged.
    - adv_dir (str): Directory containing advanced stats CSV files.
    - stats_dir (str): Directory containing traditional stats CSV files.
    - output_dir (str): Directory to save the merged output.
    """
    # File paths
    adv_file = os.path.join(adv_dir, f"player-adv-{year}.csv")
    stats_file = os.path.join(stats_dir, f"stats_{year}.csv")
    output_file = os.path.join(output_dir, f"players_{year}.csv")

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Load the CSV files
    if not os.path.exists(adv_file) or not os.path.exists(stats_file):
        print(f"Files missing for year {year}. Skipping merge...")
        return

    adv_df = pd.read_csv(adv_file, encoding='utf-8')
    stats_df = pd.read_csv(stats_file, encoding='utf-8')

    stats_df['PLAYER_FULLNAME'] = stats_df['PLAYER_FULLNAME'].apply(fix_encoding)
    adv_df['Player'] = adv_df['Player'].apply(fix_encoding)

    # Clean and standardize data
    adv_df['Player'] = adv_df['Player'].str.upper()
    stats_df['PLAYER_FULLNAME'] = stats_df['PLAYER_FULLNAME'].str.upper()
    adv_df.rename(columns={
        'Player': 'PLAYER_FULLNAME',
        'Age': 'PLAYER_AGE',
        'Team': 'TEAM_ABBREVIATION',
        'MP': 'MIN',
        'G': 'GP'
    }, inplace=True)

    # Handle duplicates in advanced stats
    numeric_columns = adv_df.select_dtypes(include=['float64', 'int64']).columns.difference(['Rk', 'GP', 'MIN'])
    adv_df = (
        adv_df.groupby(['PLAYER_FULLNAME', 'Rk'], as_index=False)
        .agg({
            'TEAM_ABBREVIATION': 'last',  # Use the last index for team
            'GP': 'first',  # Use the first index for games played
            'MIN': lambda x: x.sum() / 2,  # Sum and divide MIN by 2
            **{col: 'mean' for col in numeric_columns}  # Average all other numeric stats
        })
    )

    # Convert relevant columns to ensure compatibility
    adv_df['PLAYER_AGE'] = adv_df['PLAYER_AGE'].astype(float)
    stats_df['PLAYER_AGE'] = stats_df['PLAYER_AGE'].astype(float)

    # Merge datasets on Games Played
    merged_df = pd.merge(
        stats_df,
        adv_df,
        on=['PLAYER_FULLNAME'],
        how='outer',
        suffixes=('_stats', '_adv')
    )

    # Resolve column conflicts
    for column in ['PLAYER_AGE', 'TEAM_ABBREVIATION', 'WS']:
        if f"{column}_adv" in merged_df.columns and f"{column}_stats" in merged_df.columns:
            merged_df[column] = merged_df[f"{column}_adv"].combine_first(merged_df[f"{column}_stats"])
            merged_df.drop(columns=[f"{column}_stats", f"{column}_adv"], inplace=True)

    merged_df.dropna(inplace=True)
    merged_df = merged_df.loc[:, ~merged_df.columns.str.contains('_stats')]
    merged_df.rename(columns=lambda col: col.replace('_adv', ''), inplace=True)
    
    # Save merged data
    merged_df.to_csv(output_file, index=False)
    print(f"Merged data saved to {output_file}")

for year in range(1980,2024):
    merge_player_stats(
        year=year,
        adv_dir="/Users/cb/src/nba_mvp_ml/data/raw/basketball-reference/player-adv-stats",
        stats_dir="/Users/cb/src/nba_mvp_ml/data/processed/by_season/players-deprecated",
        output_dir="/Users/cb/src/nba_mvp_ml/data/processed/by_season/players"
    )

Merged data saved to /Users/cb/src/nba_mvp_ml/data/processed/by_season/players/players_1980.csv
Merged data saved to /Users/cb/src/nba_mvp_ml/data/processed/by_season/players/players_1981.csv
Merged data saved to /Users/cb/src/nba_mvp_ml/data/processed/by_season/players/players_1982.csv
Merged data saved to /Users/cb/src/nba_mvp_ml/data/processed/by_season/players/players_1983.csv
Merged data saved to /Users/cb/src/nba_mvp_ml/data/processed/by_season/players/players_1984.csv
Merged data saved to /Users/cb/src/nba_mvp_ml/data/processed/by_season/players/players_1985.csv
Merged data saved to /Users/cb/src/nba_mvp_ml/data/processed/by_season/players/players_1986.csv
Merged data saved to /Users/cb/src/nba_mvp_ml/data/processed/by_season/players/players_1987.csv
Merged data saved to /Users/cb/src/nba_mvp_ml/data/processed/by_season/players/players_1988.csv
Merged data saved to /Users/cb/src/nba_mvp_ml/data/processed/by_season/players/players_1989.csv
Merged data saved to /Users/cb/src/nba_m